In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

In [10]:
data_path = r"D:data/startup/diabetic_data.csv"
map_path = r"D:data/startup/IDS_mapping.csv"

In [20]:
df= pd.read_csv(data_path)
maps= pd.read_csv(map_path, dtype=str) 

print("df:", df.shape)
print("maps:", maps.shape)

df: (101766, 50)
maps: (67, 2)


In [21]:
df=df.replace("?", np.nan)

In [22]:
df.head(2)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30


In [23]:
maps.head(20)

,admission_type_id,description
0,1,Emergency
1,2,Urgent
2,3,Elective
3,4,Newborn
4,5,Not Available
5,6,NaN
6,7,Trauma Center
7,8,Not Mapped
8,NaN,NaN
9,discharge_disposition_id,description


In [25]:
# Parsing the stacked map file into three clean tables
maps.columns = [c.strip().lower() for c in maps.columns]

# separating id and description columns
id_col = maps.columns[0]
desc_col = maps.columns[1]

m = maps[[id_col, desc_col]].copy()
m[id_col] = m[id_col].astype(str).str.strip()
m[desc_col] = m[desc_col].astype(str).str.strip()

# Section headers that appear inside the first column
sections = {"admission_type_id", "discharge_disposition_id", "admission_source_id"}

# Creating a 'section' column that forward-fills after each header row
m["section"] = np.where(m[id_col].isin(sections), m[id_col], np.nan)
m["section"] = m["section"].ffill()

# top rows (admission_type) have no header row, so fill them as admission_type_id
m["section"] = m["section"].fillna("admission_type_id")

# Converting rows that are actual numeric IDs
m["id_num"] = pd.to_numeric(m[id_col], errors="coerce")

# Keeping only valid mapping rows
m2 = m[m["id_num"].notna() & m["section"].isin(sections)].copy()

# Dropping description rows 
m2.loc[m2[desc_col].str.lower().isin(["nan", "none", ""]), desc_col] = np.nan
m2 = m2.dropna(subset=[desc_col])

# Build mapping tables
adm_map = m2[m2["section"] == "admission_type_id"][["id_num", desc_col]].rename(
    columns={"id_num": "admission_type_id", desc_col: "admission_type"}
)

dis_map = m2[m2["section"] == "discharge_disposition_id"][["id_num", desc_col]].rename(
    columns={"id_num": "discharge_disposition_id", desc_col: "discharge_disposition"}
)

src_map = m2[m2["section"] == "admission_source_id"][["id_num", desc_col]].rename(
    columns={"id_num": "admission_source_id", desc_col: "admission_source"}
)

print("adm_map:", adm_map.shape)
print("dis_map:", dis_map.shape)
print("src_map:", src_map.shape)

adm_map.head(10), dis_map.head(5), src_map.head(5)


adm_map: (7, 2)
dis_map: (29, 2)
src_map: (24, 2)


(   admission_type_id admission_type
 0                1.0      Emergency
 1                2.0         Urgent
 2                3.0       Elective
 3                4.0        Newborn
 4                5.0  Not Available
 6                7.0  Trauma Center
 7                8.0     Not Mapped,
     discharge_disposition_id  \
 10                       1.0   
 11                       2.0   
 12                       3.0   
 13                       4.0   
 14                       5.0   
 
                                 discharge_disposition  
 10                                 Discharged to home  
 11  Discharged/transferred to another short term h...  
 12                      Discharged/transferred to SNF  
 13                      Discharged/transferred to ICF  
 14  Discharged/transferred to another type of inpa...  ,
     admission_source_id                                admission_source
 42                  1.0                              Physician Referral
 43           

In [26]:
df["admission_type_id"] = pd.to_numeric(df["admission_type_id"], errors="coerce")
df["discharge_disposition_id"] = pd.to_numeric(df["discharge_disposition_id"], errors="coerce")
df["admission_source_id"] = pd.to_numeric(df["admission_source_id"], errors="coerce")

df = df.merge(adm_map, on="admission_type_id", how="left")
df = df.merge(dis_map, on="discharge_disposition_id", how="left")
df = df.merge(src_map, on="admission_source_id", how="left")


In [28]:
[c for c in df.columns if "admission" in c.lower() or "type" in c.lower() or "description" in c.lower()]


['admission_type_id',
 'admission_source_id',
 'admission_type_x',
 'admission_source_x',
 'admission_type_y',
 'admission_source_y']

In [29]:
df[["admission_type_x","admission_type_y"]].head(10)

,admission_type_x,admission_type_y
0,NaN,NaN
1,Emergency,Emergency
2,Emergency,Emergency
3,Emergency,Emergency
4,Emergency,Emergency
5,Urgent,Urgent
6,Elective,Elective
7,Emergency,Emergency
8,Urgent,Urgent
9,Elective,Elective


In [30]:
# Renameing to clean names
df = df.rename(columns={
    "admission_type_y": "admission_type",
    "admission_source_y": "admission_source"
})

# Dropping extra columns
df = df.drop(columns=[
    "admission_type_x",
    "admission_type_y",
    "admission_source_x",
    "admission_source_y"
], errors="ignore")

print(df[["admission_type_id","admission_type"]].drop_duplicates().head(10))
print(df[["admission_source_id","admission_source"]].drop_duplicates().head(10))


       admission_type_id admission_type
0                      6            NaN
1                      1      Emergency
5                      2         Urgent
6                      3       Elective
2043                   4        Newborn
3089                   5  Not Available
7789                   8     Not Mapped
45829                  7  Trauma Center
      admission_source_id                                admission_source
0                       1                              Physician Referral
1                       7                                  Emergency Room
5                       2                                 Clinic Referral
8                       4                        Transfer from a hospital
335                     5  Transfer from a Skilled Nursing Facility (SNF)
396                     6      Transfer from another health care facility
469                    20                                      Not Mapped
1014                    3                       

In [34]:
[c for c in df.columns if "admission" in c.lower() or "discharge" in c.lower()]


['admission_type_id',
 'discharge_disposition_id',
 'admission_source_id',
 'discharge_disposition_x',
 'admission_type_x',
 'discharge_disposition_y',
 'admission_source_x',
 'admission_type_y',
 'discharge_disposition',
 'admission_source_y']

In [35]:
# Create admission_type from whatever exists
if "admission_type" not in df.columns:
    candidates = [c for c in df.columns if c.lower() in ["admission_type_x", "admission_type_y"]]
    if candidates:
        # pick _y if available else first one
        pick = "admission_type_y" if "admission_type_y" in candidates else candidates[0]
        df["admission_type"] = df[pick]
    else:
        print("No admission_type mapping columns found.")

# Same for admission_source
if "admission_source" not in df.columns:
    candidates = [c for c in df.columns if c.lower() in ["admission_source_x", "admission_source_y"]]
    if candidates:
        pick = "admission_source_y" if "admission_source_y" in candidates else candidates[0]
        df["admission_source"] = df[pick]
    else:
        print("No admission_source mapping columns found.")

# Same for discharge_disposition
if "discharge_disposition" not in df.columns:
    candidates = [c for c in df.columns if c.lower() in ["discharge_disposition_x", "discharge_disposition_y"]]
    if candidates:
        pick = "discharge_disposition_y" if "discharge_disposition_y" in candidates else candidates[0]
        df["discharge_disposition"] = df[pick]
    else:
        print("No discharge_disposition mapping columns found.")

# Fill missing
for c in ["admission_type", "admission_source", "discharge_disposition"]:
    if c in df.columns:
        df[c] = df[c].fillna("Unknown")

# Validate
print(df[["admission_type_id", "admission_type"]].drop_duplicates().head(10))
print(df[["admission_source_id", "admission_source"]].drop_duplicates().head(10))
print(df[["discharge_disposition_id", "discharge_disposition"]].drop_duplicates().head(10))


       admission_type_id admission_type
0                      6        Unknown
1                      1      Emergency
5                      2         Urgent
6                      3       Elective
2043                   4        Newborn
3089                   5  Not Available
7789                   8     Not Mapped
45829                  7  Trauma Center
      admission_source_id                                admission_source
0                       1                              Physician Referral
1                       7                                  Emergency Room
5                       2                                 Clinic Referral
8                       4                        Transfer from a hospital
335                     5  Transfer from a Skilled Nursing Facility (SNF)
396                     6      Transfer from another health care facility
469                    20                                      Not Mapped
1014                    3                       

In [36]:
OUT = r"D:\data\startup\master_readmission_dataset.csv"
df.to_csv(OUT, index=False)
print("Saved clean master:", OUT, "| cols:", df.shape[1])


Saved clean master: D:\data\startup\master_readmission_dataset.csv | cols: 60


In [42]:
# Drop duplicates
cols_to_drop = [
    "admission_type_x","admission_type_y",
    "admission_source_x","admission_source_y",
    "discharge_disposition_x","discharge_disposition_y"
]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors="ignore")

# Quick check
print([c for c in df.columns if c.endswith("_x") or c.endswith("_y")])

# Resaving master
OUT = r"D:\data\startup\master_readmission_dataset.csv"
df.to_csv(OUT, index=False)
print("Saved clean master:", OUT, "| cols:", df.shape[1])


[]
Saved clean master: D:\data\startup\master_readmission_dataset.csv | cols: 54


In [43]:
demo = pd.read_csv("d:data/startup/master_readmission_dataset.csv")

C:\Users\syeds\AppData\Local\Temp\ipykernel_19996\2167777518.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  demo = pd.read_csv("d:data/startup/master_readmission_dataset.csv")


In [44]:
demo.shape

(101766, 54)

In [45]:
demo.columns


Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'readmit_30', 'discharge_disposition', 'admission_type',
       'admission_source'],


In [48]:
demo.head(20)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,readmit_30,discharge_disposition,admission_type,admission_source
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,NO,0,Not Mapped,Unknown,Physician Referral
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,No,No,Ch,Yes,>30,0,Discharged to home,Emergency,Emergency Room
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,Yes,NO,0,Discharged to home,Emergency,Emergency Room
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,No,No,Ch,Yes,NO,0,Discharged to home,Emergency,Emergency Room
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,No,No,Ch,Yes,NO,0,Discharged to home,Emergency,Emergency Room
5,35754,82637451,Caucasian,Male,[50-60),NaN,2,1,2,3,...,No,No,No,No,Yes,>30,0,Discharged to home,Urgent,Clinic Referral
6,55842,84259809,Caucasian,Male,[60-70),NaN,3,1,2,4,...,No,No,No,Ch,Yes,NO,0,Discharged to home,Elective,Clinic Referral
7,63768,114882984,Caucasian,Male,[70-80),NaN,1,1,7,5,...,No,No,No,No,Yes,>30,0,Discharged to home,Emergency,Emergency Room
8,12522,48330783,Caucasian,Female,[80-90),NaN,2,1,4,13,...,No,No,No,Ch,Yes,NO,0,Discharged to home,Urgent,Transfer from a hospital
9,15738,63555939,Caucasian,Female,[90-100),NaN,3,3,4,12,...,No,No,No,Ch,Yes,NO,0,Discharged/transferred to SNF,Elective,Transfer from a hospital
